## Custom Agent to Output Tx Params using Tool


In [1]:
from graph.converter_tool import generate_tx_params

from_address = "0xd8da6bf26964af9d7eed9e03e53415d37aa96045"

In [2]:
description = "Send 0.01 ETH to the address 0x8c575b178927fF9A70804B8b4F7622F7666bB360."
res = await generate_tx_params(description, from_address)
print(res)

assert res.from_address.lower() == "0xd8da6bf26964af9d7eed9e03e53415d37aa96045".lower()
assert res.to_address.lower() == "0x8c575b178927fF9A70804B8b4F7622F7666bB360".lower()
assert res.data == "0x"
assert res.value == "0x2386f26fc10000"

From: 0xd8da6bf26964af9d7eed9e03e53415d37aa96045
To: 0x8c575b178927fF9A70804B8b4F7622F7666bB360
Data: 0x
Value: 0x2386f26fc10000


In [3]:
description = "Transfer 100 USDT to Scott's Ethereum address."
res = await generate_tx_params(description, from_address)
print(res)

assert res.from_address.lower() == "0xd8da6bf26964af9d7eed9e03e53415d37aa96045".lower()
assert res.to_address.lower() == "0xdAC17F958D2ee523a2206206994597C13D831ec7".lower()
data = "0xa9059cbb000000000000000000000000b859d7eb5e319e54dc23c83dfe93e649b88202b60000000000000000000000000000000000000000000000000000000005f5e100"
assert res.data == data
assert res.value == "0x0"

From: 0xd8da6bf26964af9d7eed9e03e53415d37aa96045
To: 0xdac17f958d2ee523a2206206994597c13d831ec7
Data: 0xa9059cbb000000000000000000000000b859d7eb5e319e54dc23c83dfe93e649b88202b60000000000000000000000000000000000000000000000000000000005f5e100
Value: 0x0


In [4]:
description = "Approve 100 USDT for swapping on a decentralized exchange."
res = await generate_tx_params(description, from_address)
print(res)

assert res.from_address.lower() == "0xd8da6bf26964af9d7eed9e03e53415d37aa96045".lower()
assert res.to_address.lower() == "0xdAC17F958D2ee523a2206206994597C13D831ec7".lower()
data = "0x095ea7b30000000000000000000000007a250d5630b4cf539739df2c5dacb4c659f2488d0000000000000000000000000000000000000000000000000000000005f5e100"
assert res.data == data
assert res.value == "0x0"

From: 0xd8da6bf26964af9d7eed9e03e53415d37aa96045
To: 0xdac17f958d2ee523a2206206994597c13d831ec7
Data: 0x095ea7b30000000000000000000000007a250d5630b4cf539739df2c5dacb4c659f2488d0000000000000000000000000000000000000000000000000000000005f5e100
Value: 0x0


In [5]:
description = "Swap 100 USDT to USDC on a decentralized exchange."
res = await generate_tx_params(description, from_address)
print(res)

assert res.from_address.lower() == "0xd8da6bf26964af9d7eed9e03e53415d37aa96045".lower()
assert res.to_address.lower() == "0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D".lower()

# Function selector (UniswapV2 Router: swapExactTokensForTokens)
assert res.data[:10] == "0x38ed1739"
# Amount of input tokens to send
amount_input = "0000000000000000000000000000000000000000000000000000000005f5e100"
assert res.data[10:74] == amount_input
# Amount of output tokens to receive
amount_output = "0000000000000000000000000000000000000000000000000000000000000000"
assert res.data[74:138] == amount_output
# Address of the recipient
recipient = "d8da6bf26964af9d7eed9e03e53415d37aa96045"
assert res.data[226:266] == recipient

import time

# Deadline for the transaction
deadline_hex = res.data[266:330]
deadline = int(deadline_hex, 16)
assert deadline > int(time.time()) + 30  # 30 seconds from now
print(f"Seconds from now: {deadline - int(time.time())}")

# First address in the path (USDT)
token1_address = "dac17f958d2ee523a2206206994597c13d831ec7"
assert res.data[418:458] == token1_address
# Second address in the path (USDC)
token2_address = "a0b86991c6218b36c1d19d4a2e9eb0ce3606eb48"
assert res.data[482:522] == token2_address

assert res.value == "0x0"

From: 0xd8dA6BF26964aF9D7eEd9e03E53415D37aA96045
To: 0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D
Data: 0x38ed17390000000000000000000000000000000000000000000000000000000005f5e100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000a0000000000000000000000000d8da6bf26964af9d7eed9e03e53415d37aa960450000000000000000000000000000000000000000000000000000000067027ff40000000000000000000000000000000000000000000000000000000000000002000000000000000000000000dac17f958d2ee523a2206206994597c13d831ec7000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
Value: 0x0
Seconds from now: 3583


In [6]:
description = "Stake 0.03 ETH to Lido to receive stETH."
res = await generate_tx_params(description, from_address)
print(res)

assert res.from_address.lower() == "0xd8da6bf26964af9d7eed9e03e53415d37aa96045".lower()
assert res.to_address.lower() == "0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84".lower()
data = "0xa1903eab0000000000000000000000000000000000000000000000000000000000000000"
assert res.data == data
assert res.value == "0x6a94d74f430000"

From: 0xd8da6bf26964af9d7eed9e03e53415d37aa96045
To: 0xae7ab96520de3a18e5e111b5eaab095312d7fe84
Data: 0xa1903eab0000000000000000000000000000000000000000000000000000000000000000
Value: 0x6a94d74f430000


In [7]:
description = "Stake 0.299999999999999997 stETH to Eigenpie."
res = await generate_tx_params(description, from_address)
print(res)

From: 0xd8da6bf26964af9d7eed9e03e53415d37aa96045
To: 0x24db6717dB1C75B9Db6eA47164D8730B63875dB7
Data: 0x2ebe07c8000000000000000000000000ae7ab96520de3a18e5e111b5eaab095312d7fe840000000000000000000000000000000000000000000000000429d069189dfffd00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
Value: 0x0


## Custom Agent to Output Tx Params using LLM


In [8]:
from graph.converter_llm import generate_tx_params

from_address = "0xd8da6bf26964af9d7eed9e03e53415d37aa96045"

In [9]:
description = "Send 0.01 ETH to the address 0x8c575b178927fF9A70804B8b4F7622F7666bB360."
res = await generate_tx_params(description, from_address)
print(res.model_dump_json())

assert res.from_address.lower() == "0xd8da6bf26964af9d7eed9e03e53415d37aa96045".lower()
assert res.to_address.lower() == "0x8c575b178927fF9A70804B8b4F7622F7666bB360".lower()
assert res.data == "0x"
assert res.value == "0x2386f26fc10000"

{"from_address":"0xd8da6bf26964af9d7eed9e03e53415d37aa96045","to_address":"0x8c575b178927fF9A70804B8b4F7622F7666bB360","data":"0x","value":"0x2386f26fc10000"}


In [10]:
description = "Transfer 100 USDT to Scott's Ethereum address."
res = await generate_tx_params(description, from_address)
print(res.model_dump_json())

assert res.from_address.lower() == "0xd8da6bf26964af9d7eed9e03e53415d37aa96045".lower()
assert res.to_address.lower() == "0xdAC17F958D2ee523a2206206994597C13D831ec7".lower()
data = "0xa9059cbb000000000000000000000000b859d7eb5e319e54dc23c83dfe93e649b88202b60000000000000000000000000000000000000000000000000000000005f5e100"
assert res.data == data
assert res.value == "0x0"

{"from_address":"0xd8da6bf26964af9d7eed9e03e53415d37aa96045","to_address":"0xdAC17F958D2ee523a2206206994597C13D831ec7","data":"0xa9059cbb000000000000000000000000b859d7eb5e319e54dc23c83dfe93e649b88202b60000000000000000000000000000000000000000000000000000000005f5e100","value":"0x0"}


In [11]:
description = "Approve 100 USDT for swapping on a decentralized exchange."
res = await generate_tx_params(description, from_address)
print(res.model_dump_json())

assert res.from_address.lower() == "0xd8da6bf26964af9d7eed9e03e53415d37aa96045".lower()
assert res.to_address.lower() == "0xdAC17F958D2ee523a2206206994597C13D831ec7".lower()
data = "0x095ea7b30000000000000000000000007a250d5630b4cf539739df2c5dacb4c659f2488d0000000000000000000000000000000000000000000000000000000005f5e100"
assert res.data == data
assert res.value == "0x0"

{"from_address":"0xd8da6bf26964af9d7eed9e03e53415d37aa96045","to_address":"0xdac17f958d2ee523a2206206994597c13d831ec7","data":"0x095ea7b30000000000000000000000007a250d5630b4cf539739df2c5dacb4c659f2488d0000000000000000000000000000000000000000000000000000000005f5e100","value":"0x0"}


In [12]:
description = "Swap 100 USDT to USDC on a decentralized exchange."
res = await generate_tx_params(description, from_address)
print(res.model_dump_json())

assert res.from_address.lower() == "0xd8da6bf26964af9d7eed9e03e53415d37aa96045".lower()
assert res.to_address.lower() == "0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D".lower()
# Data is omitted since deadline is not deterministic
assert res.value == "0x0"

{"from_address":"0xd8da6bf26964af9d7eed9e03e53415d37aa96045","to_address":"0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D","data":"0x38ed17390000000000000000000000000000000000000000000000000000000005f5e100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000a0000000000000000000000000d8da6bf26964af9d7eed9e03e53415d37aa96045000000000000000000000000000000000000000000000000000000006702803d0000000000000000000000000000000000000000000000000000000000000002000000000000000000000000dac17f958d2ee523a2206206994597c13d831ec7000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb48","value":"0x0"}


In [13]:
description = "Stake 0.03 ETH to Lido to receive stETH."
res = await generate_tx_params(description, from_address)
print(res.model_dump_json())

assert res.from_address.lower() == "0xd8da6bf26964af9d7eed9e03e53415d37aa96045".lower()
assert res.to_address.lower() == "0xae7ab96520DE3A18E5e111B5EaAb095312D7fE84".lower()
data = "0xa1903eab0000000000000000000000000000000000000000000000000000000000000000"
assert res.data == data
assert res.value == "0x6a94d74f430000"

{"from_address":"0xd8da6bf26964af9d7eed9e03e53415d37aa96045","to_address":"0xae7ab96520de3a18e5e111b5eaab095312d7fe84","data":"0xa1903eab0000000000000000000000000000000000000000000000000000000000000000","value":"0x6a94d74f430000"}
